In [ ]:
##### Copyright 2021 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Calibration: Overview and API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/noise/calibration_api"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/noise/calibration_api.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/noise/calibration_api.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/noise/calibration_api.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

This tutorial provides an introductory overview of calibration. By the end, you will understand the high-level calibration process, how to run calibrations through the API, and how to choose which calibration technique to use.

## Setup
Note: this notebook relies on unreleased Cirq features. If you want to try these features, make sure you install cirq via `pip install cirq --pre`.

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq --pre
    print("installed cirq.")
    import cirq
print("Using cirq version", cirq.__version__)

0.15.0.dev


In [ ]:
import numpy as np

import cirq
import cirq_google as cg

Note: Leave the `project_id` and/or `processor_id` blank to use a noisy simulator.

In [ ]:
import os

# The Google Cloud Project id to use.
project_id = '' #@param {type:"string"}
processor_id = "" #@param {type:"string"}

from cirq_google.engine.qcs_notebook import get_qcs_objects_for_notebook
device_sampler = get_qcs_objects_for_notebook(project_id, processor_id)

## Overview

Floquet and XEB calibration are tools to characterize a family of gates including $\sqrt{\text{iSWAP}}$ gates on a particular processor and compensate for errors. The family of gates is represented by the `cirq.PhasedFSimGate`, a two-qubit gate with five angles $\theta$, $\zeta$, $\chi$, $\gamma$, $\phi$ given by the following unitary matrix:

$$
U(\theta, \zeta, \chi, \gamma, \phi) := \left[ \begin{matrix}
1 & 0                                          & 0                                         & 0      \\
0 &    \exp(-i \gamma - i \zeta) \cos( \theta ) & -i \exp(-i \gamma + i \chi) \sin( \theta ) & 0 \\
0 & -i \exp(-i \gamma - i \chi) \sin( \theta )  &    \exp(-i \gamma + i \zeta) \cos( \theta) & 0    \\
0 & 0                                          & 0                               & \exp(-2 i \gamma -i \phi )  
\end{matrix} \right]
$$

The $\sqrt{\text{iSWAP}}$ gate corresponds to angles $\theta=-\pi / 4$ and $\zeta = \chi = \gamma = \phi = 0$. 

Characterization is done by the Quantum Engine and compensation (currently the insertion of $Z$ gates around $\sqrt{\text{iSWAP}}$ gates) is completely client-side with the help of Cirq utilities. At the highest level, each calibration tool inputs one or more quantum circuits of interest as well as a processor to run on, and outputs the calibrated circuit(s) for this processor.

Both calibrations are primarily focused on decreasing the effects of two sources of coherent errors: calibration drifts and cross-talk errors. These errors are not uniform across the chip and vary in magnitude between qubits and device calibrations. The circuit-specific calibrations should be used in situations when achieving the best performance is more important than using extra time on the device for the characterizations.

## Calibration techniques and API

To run calibration, define your circuit or circuits below. For sake of example we use a simple circuit with a single $\sqrt{\text{iSWAP}}$ gate.

In [ ]:
"""Define your circuit(s) here."""
qubits = cg.line_on_device(device_sampler.device, length=2)
circuit = cirq.Circuit(cirq.ISWAP.on(*qubits) ** 0.5)

print("Example circuit:", circuit, sep="\n\n")

Note: Above we picked any qubits, but in an actual experiment you want to pick a good set of qubits to run on. To do so, use the latest maintanence report as in [this XEB calibration example](./qcvv/xeb_calibration_example.ipynb) or use a Loschmidt echo as in [this Loschmidt echo tutorial](../tutorials/google/echoes.ipynb).

### Floquet calibration

Floquet calibration returns fast, accurate estimates for every angle of the `cirq.PhasedFSimGate` except $\chi$ (chi).

The first step to Floquet calibration is defining the angles to characterize.

In [ ]:
# Define calibration options.
floquet_options = cg.FloquetPhasedFSimCalibrationOptions(
    characterize_theta=True,
    characterize_zeta=True,
    characterize_chi=False,
    characterize_gamma=True,
    characterize_phi=True,
)

Next, use these options to get characterization requests - i.e., `cirq.Moment`s to characterize on the Quantum Engine - then characterize them.

In [ ]:
# Get characterization requests.
floquet_characterization_requests = cg.prepare_characterization_for_operations(
    circuit, options=floquet_options
)

# Characterize the requests on the engine.
floquet_characterizations = cg.run_calibrations(
    floquet_characterization_requests, device_sampler.sampler
)

Note: The first argument to `cirq_google.prepare_characterization_for_operations` can also be a sequence of circuits.

The last step is to compensate for offsets using the `cirq_google.make_zeta_chi_gamma_compensation_for_moments` function.

In [ ]:
# Compensate your circuit based on the characterizations.
floquet_calibrated_circuit = cg.make_zeta_chi_gamma_compensation_for_moments(
    circuit, floquet_characterizations
).circuit

print("Floquet calibrated circuit:", floquet_calibrated_circuit, sep="\n\n")

Note the addition of `cirq.Rz` gates to compensate for characterized errors. This calibrated circuit can now be run on the processor.

### XEB calibration

XEB calibration works by executing a library of random circuits both on a noiseless simulator and on a noisy processor, then comparing results. In principle it can characterize any two-qubit gate, but compensation is currently only for $\zeta$, $\chi$, and $\gamma$. For more details about how XEB calibration works, see the [Parallel XEB tutorial](./qcvv/parallel_xeb.ipynb).

Like Floquet calibration, the first step to using XEB calibration is defining options. Unlike Floquet calibration, XEB calibration requires significant classical processing to characterize angles. In addition to the angles to characterize, we also need to define the `cycle_depths` of the random circuits. The number of processes `n_processes` is a parallelization option for the classical processing.

In [ ]:
# Define calibration options.
cycle_depths=(5, 25, 50, 100)

xeb_options = cg.LocalXEBPhasedFSimCalibrationOptions(
    cycle_depths=cycle_depths,
    n_processes=1,
    # Note that all angles below are set to True by default.
    fsim_options=cirq.experiments.XEBPhasedFSimCharacterizationOptions(
        characterize_theta=False,
        characterize_zeta=True,
        characterize_chi=True,
        characterize_gamma=True,
        characterize_phi=False,
    ),
)

The remaining steps are identical to Floquet calibration: first getting characterization requests then characterizing them on the engine.

In [ ]:
# Get characterization requests.
xeb_characterization_requests = cg.prepare_characterization_for_operations(
    circuit, options=xeb_options
)

# Characterize the requests on the engine.
xeb_characterizations = cg.run_calibrations(
    xeb_characterization_requests, device_sampler.sampler
)

Making compensations is also identical to the Floquet calibration example.

In [ ]:
xeb_calibrated_circuit = cg.make_zeta_chi_gamma_compensation_for_moments(
    circuit, xeb_characterizations,
).circuit

print("XEB calibrated circuit:", xeb_calibrated_circuit, sep="\n\n")

Again note the addition of `cirq.Rz` gates to compensate for errors. This calibrated circuit can now be run on the processor.

## Which calibration should I use?

This question is very dependent on the particular type of circuit, but some guidelines exist. XEB calibration is generally more applicable to circuits that alternate single-qubit and two-qubit layers due to the similarity between the random XEB circuits and executed circuits. Floquet calibration may be more applicable to circuits with two-qubit gates and $Z$ gates only.

## Summary and notes

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-c3ow"></th>
    <th class="tg-c3ow"><span style="font-weight:bold">Floquet calibration</span></th>
    <th class="tg-c3ow"><span style="font-weight:bold">XEB calibration</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-c3ow"><span style="font-weight:bold">FSim angles characterized</span></td>
    <td class="tg-c3ow">4 / 5 (all but χ)</td>
    <td class="tg-c3ow">5 / 5</td>
  </tr>
  <tr>
    <td class="tg-7btt"><span style="font-weight:bold">FSim angles compensated</span></td>
    <td class="tg-c3ow">2 / 5 (ζ and γ)</td>
    <td class="tg-c3ow">3 / 5 (ζ, χ, and γ)<br></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold">Relative runtime</span></td>
    <td class="tg-baqh">1</td>
    <td class="tg-baqh">~10x Floquet calibration runtime</td>
  </tr>
  <tr>
    <td class="tg-c3ow"><span style="font-weight:bold;font-style:normal">Robust to SPAM errors?</span></td>
    <td class="tg-c3ow">Yes</td>
    <td class="tg-c3ow">Yes</td>
  </tr>
  <tr>
    <td class="tg-7btt"><span style="font-weight:bold;font-style:normal">Experiments used in</span></td>
    <td class="tg-c3ow"><a href="https://arxiv.org/abs/2010.07965" target="_blank" rel="noopener noreferrer">Fermi-Hubbard</a></td>
    <td class="tg-c3ow"><a href="https://arxiv.org/abs/2101.08870" target="_blank" rel="noopener noreferrer">OTOC</a></td>
  </tr>
</tbody>
</table>

- The ζ, χ and γ parameters of the `cirq.PhasedFSimGate` are subject to drifts that range in frequency from seconds to hours.
- Neither calibration technique attempts to correct the misaligned iSWAP rotation or the additional two-qubit phase. This is a non-trivial task and we do currently have simple tools to achieve this. It is up to the user to correct for these in experiments as best as possible.
- Floquet calibration does not yet support microwave gates.

## Next steps

Try running Floquet and/or XEB calibration on your circuit(s). See the following tutorials for detailed examples and benchmarks.

- [Floquet calibration: Example and benchmark](./floquet_calibration_example.ipynb)
- [XEB calibration: Example and benchmark](./qcvv/xeb_calibration_example.ipynb)

See also the [Calibration FAQ](./calibration_faq.md) for additional information on calibration.